## CBoW မော်ဒယ်ကိုလေ့ကျင့်ခြင်း

ဒီ notebook က [AI for Beginners Curriculum](http://aka.ms/ai-beginners) ရဲ့ အစိတ်အပိုင်းတစ်ခုဖြစ်ပါတယ်။

ဒီဥပမာမှာတော့ CBoW ဘာသာစကားမော်ဒယ်ကိုလေ့ကျင့်ပြီး Word2Vec embedding space ကို ကိုယ်တိုင်ရရှိအောင်လုပ်မယ်။ AG News dataset ကို စာသားရင်းမြစ်အဖြစ်အသုံးပြုမယ်။


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

အရင်ဆုံး ကျွန်တော်တို့ရဲ့ ဒေတာစနစ်ကို load လုပ်ပြီး tokenizer နဲ့ vocabulary ကို သတ်မှတ်ပါမယ်။ `vocab_size` ကို ၅၀၀၀ အထိ သတ်မှတ်ပြီး ရှုထောင့်များကို အနည်းငယ် ကန့်သတ်ပါမယ်။


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW မော်ဒယ်

CBoW သည် $2N$ အနီးအနားရှိ စကားလုံးများကို အခြေခံပြီး စကားလုံးတစ်လုံးကို ခန့်မှန်းရန် သင်ယူသည်။ ဥပမာအားဖြင့် $N=1$ ဖြစ်သောအခါ *I like to train networks* ဆိုသော စာကြောင်းမှ အောက်ပါ စုံတွဲများကို ရရှိမည်ဖြစ်သည် - (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks)။ ဒီမှာ ပထမ စကားလုံးသည် input အဖြစ် အသုံးပြုသော အနီးအနားရှိ စကားလုံးဖြစ်ပြီး ဒုတိယ စကားလုံးသည် ခန့်မှန်းရန် ရည်ရွယ်ထားသော စကားလုံးဖြစ်သည်။

နောက်ထပ် စကားလုံးကို ခန့်မှန်းရန် network တစ်ခုကို တည်ဆောက်ရန်အတွက် အနီးအနားရှိ စကားလုံးကို input အဖြစ် ပေးရမည်ဖြစ်ပြီး စကားလုံးနံပါတ်ကို output အဖြစ် ရယူရမည်။ CBoW network ၏ architecture သည် အောက်ပါအတိုင်းဖြစ်သည် -

* Input စကားလုံးကို embedding layer မှတဆင့် ဖြတ်သွားမည်။ ဒီ embedding layer သည် Word2Vec embedding ဖြစ်မည်ဖြစ်ပြီး `embedder` variable အဖြစ် သီးသန့်သတ်မှတ်မည်။ ဤဥပမာတွင် embedding size = 30 ကို အသုံးပြုမည်ဖြစ်သော်လည်း အမြင့် dimension များ (အမှန်တကယ် Word2Vec တွင် 300 ရှိသည်) ကို စမ်းသပ်လိုပါက ပြောင်းလဲနိုင်သည်။
* Embedding vector ကို linear layer သို့ ဖြတ်သွားပြီး output စကားလုံးကို ခန့်မှန်းမည်။ ထို့ကြောင့် `vocab_size` neurons ရှိသည်။

Output အတွက် `CrossEntropyLoss` ကို loss function အဖြစ် အသုံးပြုပါက one-hot encoding မပါဘဲ စကားလုံးနံပါတ်များကိုသာ expected results အဖြစ် ပေးရမည်။


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## လေ့ကျင့်မှု ဒေတာ ပြင်ဆင်ခြင်း

အခုတော့ စာသားထဲမှ CBoW စကားလုံးစုံများကိုတွက်ချက်ပေးမည့် အဓိက function ကိုရေးကြမယ်။ ဒီ function က window size ကို သတ်မှတ်ပေးနိုင်မှာဖြစ်ပြီး၊ input နှင့် output စကားလုံးစုံများကို ပြန်ပေးမယ်။ ဒီ function ကို စကားလုံးများအပေါ်မှာသာမက၊ vectors/tensors အပေါ်မှာလည်း အသုံးပြုနိုင်တယ် - ဒါကြောင့် စာသားကို encode လုပ်ပြီးနောက်မှ `to_cbow` function ကို ပေးပို့နိုင်မယ်။


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


ကျောင်းသားသင်ကြားမှုဒေတာစုံကို ပြင်ဆင်ကြမယ်။ သတင်းအားလုံးကို ကျော်လွှားပြီး `to_cbow` ကို ခေါ်သုံးကာ စကားလုံးအတွဲများစာရင်းကို ရယူပြီး အဲ့ဒီအတွဲများကို `X` နဲ့ `Y` ထဲထည့်ပါမယ်။ အချိန်သက်သာစေရန်အတွက် ပထမ 10k သတင်းပစ္စည်းများကိုသာ စဉ်းစားပါမယ် - သင့်မှာ စောင့်ဆိုင်းဖို့ အချိန်ပိုရှိပြီး ပိုမိုကောင်းမွန်တဲ့ embedding များရယူလိုပါက အကန့်အသတ်ကို လွယ်ကူစွာ ဖယ်ရှားနိုင်ပါတယ် :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

ကျွန်ုပ်တို့သည် အဆိုပါဒေတာကို တစ်ခုတည်းသောဒေတာစုံအဖြစ် ပြောင်းလဲမည်ဖြစ်ပြီး၊ dataloader တစ်ခုကို ဖန်တီးမည်:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

ကျွန်ုပ်တို့သည် အဆိုပါဒေတာကို တစ်ခုတည်းသောဒေတာစုံအဖြစ် ပြောင်းလဲမည်ဖြစ်ပြီး၊ dataloader တစ်ခုကို ဖန်တီးမည်:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

အခုတော့ အမှန်တကယ် သင်ကြားမှုကို စတင်ကြမယ်။ ကျွန်တော်တို့ `SGD` optimizer ကို အသုံးပြုပြီး အလွန်မြင့်မားတဲ့ learning rate နဲ့ သင်ကြားပါမယ်။ သင် `Adam` ကဲ့သို့သော အခြား optimizer များကိုလည်း စမ်းသပ်ကြည့်နိုင်ပါတယ်။ အစမှာ 10 epochs အထိ သင်ကြားပါမယ် - သင်ပိုနည်းတဲ့ loss ရချင်ရင် ဒီ cell ကို ပြန်လည် run လုပ်နိုင်ပါတယ်။


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec ကို စမ်းသပ်ခြင်း

Word2Vec ကို အသုံးပြုရန်၊ ကျွန်ုပ်တို့၏ ဝေါဟာရစာရင်းထဲရှိ စကားလုံးအားလုံးနှင့် ကိုက်ညီသော ဗက်တာများကို ထုတ်ယူကြမယ်။


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Word2Vec ကို သုံးပြီး အနီးစပ်ဆုံးသော စကားလုံးများကို ရှာဖွေခြင်းသည် စိတ်ဝင်စားဖွယ်ကောင်းသည်။ အောက်ပါ function သည် ပေးထားသော input အတွက် `n` အနီးစပ်ဆုံးသော စကားလုံးများကို ပြန်လည်ပေးပါမည်။ ၎င်းတို့ကို ရှာဖွေရန် $|w_i - v|$ ၏ norm ကို တွက်ချက်ပါမည်၊ ဤနေရာတွင် $v$ သည် input စကားလုံးနှင့် ဆက်စပ်သော vector ဖြစ်ပြီး $w_i$ သည် vocabulary အတွင်းရှိ $i$-th စကားလုံး၏ encoding ဖြစ်သည်။ ထို့နောက် array ကို sort လုပ်ပြီး `argsort` ကို အသုံးပြု၍ ဆက်စပ်သော indices များကို ပြန်လည်ပေးပြီး vocabulary အတွင်းရှိ အနီးစပ်ဆုံးသော စကားလုံးများ၏ တည်နေရာကို encode လုပ်ထားသော list ၏ ပထမဆုံး `n` elements ကို ယူပါမည်။


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## အဓိကအချက်

CBoW ကဲ့သို့သော ထူးခြားသောနည်းလမ်းများကို အသုံးပြု၍ Word2Vec မော်ဒယ်ကို လေ့ကျင့်နိုင်ပါသည်။ သင်သည် အလယ်ရှိ စကားလုံးကို ပေးပြီး အနီးအနားရှိ စကားလုံးကို ခန့်မှန်းရန် လေ့ကျင့်ထားသော skip-gram မော်ဒယ်ကိုလည်း လေ့ကျင့်ကြည့်ပြီး၊ ၎င်း၏ လုပ်ဆောင်မှုကောင်းမွန်မှုကို ကြည့်ရှုနိုင်ပါသည်။



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မတိကျမှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းစာရွက်စာတမ်းကို ၎င်း၏ မူရင်းဘာသာစကားဖြင့် အာဏာတရားရှိသော အရင်းအမြစ်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားယူမှုမှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
